<a href="https://colab.research.google.com/github/rdemarqui/studies/blob/master/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes

Example adapted from the excelent tutorial given by **Future Mojo** Channel. https://www.youtube.com/watch?v=FrWvpzoQBPQ

### Conditional Probability

$P(A \cap B) = P(A) \cdot P(B|A)$

*Probability of A and B, it's equal to probability of A times probability of B given A*

### Deriving Bayes Theorem

*Given:*

$P(A \cap B) = P(A) \cdot P(B|A)$

$P(B \cap A) = P(B) \cdot P(A|B)$

*and, according to commutativity of conjuntion:*

$P(A \cap B) = P(B \cap A)$

*so, replacing:*

$P(B) \cdot P(A|B) = P(A) \cdot P(B|A)$

*dividing both sides by P(B), we have Bayes Theorem:*

$P(A|B) = \frac {P(A) \cdot P(B|A)} {P(B)}$
